In [82]:

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import regex as re
import shutil
import sqlite3
import warnings

from datetime import datetime
from pandas.core.common import SettingWithCopyWarning
from pandas.tseries.offsets import MonthEnd
from rasterstats import zonal_stats
from uszipcode import SearchEngine


warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [83]:
with open("cfg.json", "r") as jin:
    cfg = json.load(jin)

# rewrite to ensure formatting
with open("cfg.json", "w") as jout:
    json.dump(cfg, jout, indent=4)

In [84]:
#Gathering full list of US zip codes
all_zips = []
cfg["data_dir"] + cfg["all_us_zips"]
# open file and read the content in a list
with open(cfg["data_dir"] + cfg["all_us_zips"], 'r') as filehandle:
    filecontents = filehandle.readlines()

    for line in filecontents:
        # remove linebreak which is the last character of the string
        current_place = line[:-1]

        # add item to the list
        all_zips.append(current_place)

In [85]:
#Gathering zip codes which have necessary data for installation
data_zips = []

# open file and read the content in a list
with open(cfg["data_dir"] + cfg["data_zips"], 'r') as filehandle:
    filecontents = filehandle.readlines()

    for line in filecontents:
        # remove linebreak which is the last character of the string
        current_place = line[:-1]

        # add item to the list
        data_zips.append(current_place)
        
data_zips = sorted(data_zips)

In [86]:
percent_zips = len(data_zips)/len(all_zips)
percent_zips

0.4470492396813903

The following cell applies a linear connection for housing unit count between the 2010 census and 2019 American Community Survey. This would further benefit from more refined data points. For years prior to 2010, the housing count is being held constant until more accurate input data may be located.

In [89]:
search = SearchEngine(simple_zipcode=True)

def housing_dict(f):
    pop_df2 = pd.read_csv(f)
    pop_df2 = pop_df2.drop(index = 0)
    pop_df2['zipcode'] = pop_df2.NAME.apply(lambda r: r.split()[-1]).astype(int)
    pop_df2['y_2010'] = pop_df2.zipcode.apply(lambda v: search.by_zipcode(v).to_dict()['housing_units'])
    pop_df2 = pop_df2[['zipcode','y_2010','B25001_001E']].rename(columns = {'B25001_001E':'y_2019'})
    pop_df2['y_2019'] = pop_df2['y_2019'].astype(int)
    pop_df2['slope'] = ((pop_df2['y_2019'] - pop_df2['y_2010'])/(2019-2010))
    pop_df2['intercept'] = pop_df2['y_2019'] - pop_df2.slope * 2019

    hs_dict = {}
    for i in np.linspace(1998,2019,22):
        if i <= 2010:
            array = pop_df2.y_2010
        else:
            array = pop_df2.intercept + pop_df2.slope * i
        hs_dict[int(i)] =  dict(zip(pop_df2.zipcode,array))
    return hs_dict

housing_dict = housing_dict(cfg["data_dir"] + cfg["housing"])

In [92]:
with open(cfg["data_dir"] + cfg["house_p"], 'wb') as fp:
    pickle.dump(housing_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

The following cell pulls statistics from the 2010 census at the zipcode level. While useful in initial evaluations, the bulk of this information is omitted from final evauations. This is partially due to the static nature and partially due to the general lower significance versus other features.

In [62]:


def zipdata(zipcode):
    code = search.by_zipcode(zipcode)
    code = code.to_dict()
    #print(code)
    income = code["median_household_income"]
    value = code["median_home_value"]
    density = code["population_density"]
    land_area = code["land_area_in_sqmi"]
    houses = code["housing_units"]
    state = code["state"]
    return income, value, density, houses, land_area, state

In [63]:
zip_df = pd.DataFrame({'zipcode':data_zips})
zip_df['income'],zip_df['value'],zip_df['density'],zip_df['houses'],zip_df['land'], zip_df['state'] = zip(*zip_df["zipcode"].map(zipdata))


The following cell applies zip code boundaries to a NREL image file mapping a measure of irradiance known as PSM for the year 2019. This data may eventually be replaced by features obtained through work by partner GM.

In [64]:
stats = zonal_stats(cfg["data_dir"] + cfg["plotly_zips"], cfg["data_dir"] + cfg["nrel_dni"])
irradiance = [f['mean'] for f in stats]

In [65]:
with open(cfg["data_dir"] + cfg["plotly_zips"]) as f:
    zips = json.load(f)

In [66]:
zip_ids = [i['id'] for i in zips['features']]

irr_map = dict(zip(zip_ids,irradiance))

zip_df['irradiance'] = zip_df.zipcode.map(irr_map)
zip_df.head()

,zipcode,income,value,density,houses,land,state,irradiance
0,01001,58733,213000,1466,7557,11.44,MA,4.575282
1,01002,54422,338900,528,10388,55.04,MA,4.593613
2,01003,None,None,14581,5,0.71,MA,NaN
3,01005,68644,208500,115,2044,44.24,MA,4.677823
4,01007,71875,260000,278,5839,52.64,MA,4.559018


The following cell applies 2019 electricity prices to each zipcode for the relevant corresponding state. This is used in intial analysis and replaced with monthly figures for later analysis.

In [67]:
prices = pd.read_excel(cfg["data_dir"] + cfg["annual_e_price"], header=1)

prices_res_2019 = prices[(prices["Industry Sector Category"] == "Total Electric Industry") & (prices.Year == 2019)][['State','Residential']]
price_map = dict(zip(prices_res_2019.State, prices_res_2019.Residential))

zip_df['e_price'] = zip_df.state.map(price_map)
zip_df.head()

,zipcode,income,value,density,houses,land,state,irradiance,e_price
0,01001,58733,213000,1466,7557,11.44,MA,4.575282,21.92
1,01002,54422,338900,528,10388,55.04,MA,4.593613,21.92
2,01003,None,None,14581,5,0.71,MA,NaN,21.92
3,01005,68644,208500,115,2044,44.24,MA,4.677823,21.92
4,01007,71875,260000,278,5839,52.64,MA,4.559018,21.92


Speacialty contractor wage data was gathered to gain an understanding of expected costs of labor without adjustments for costs specific to solar installations. Correlation to solar installation costs may be impacted by training, familiarity or programs associated with bringing costs down. This county-based information needed to be mapped to corresponding zip codes.

In [68]:
electricians = pd.read_csv(cfg["data_dir"] + cfg["wage_data"])
electricians = electricians.rename(columns = {"Average annual wage in the specialty trade contractors industry ":"Avg_wage"})
electricians_2 = electricians.copy()
electricians = electricians[['FIPS Code','Avg_wage']].dropna().astype(int)
electricians['FIPS Code'] = electricians['FIPS Code'].astype(str).apply(lambda n: n.zfill(5))


In [69]:
zip_county = pd.read_excel(cfg["data_dir"] + cfg["zip_county"], header=0)
zip_county = zip_county[["ZIP","COUNTY"]].astype(str)

zip_county['ZIP'] = zip_county.ZIP.apply(lambda n: n.zfill(5))
zip_county['COUNTY'] = zip_county.COUNTY.apply(lambda n: n.zfill(5))

z_c = dict(zip(zip_county.ZIP,zip_county.COUNTY))
c_w = dict(zip(electricians['FIPS Code'],electricians.Avg_wage))

zip_df['county'] = zip_df.zipcode.map(z_c)
zip_df['avg_contractor_wage'] = zip_df.county.map(c_w)
zip_df.head()

,zipcode,income,value,density,houses,land,state,irradiance,e_price,county,avg_contractor_wage
0,01001,58733,213000,1466,7557,11.44,MA,4.575282,21.92,25013,65846.0
1,01002,54422,338900,528,10388,55.04,MA,4.593613,21.92,25015,53390.0
2,01003,None,None,14581,5,0.71,MA,NaN,21.92,25015,53390.0
3,01005,68644,208500,115,2044,44.24,MA,4.677823,21.92,25027,66248.0
4,01007,71875,260000,278,5839,52.64,MA,4.559018,21.92,25015,53390.0


In [70]:
zip_cluster_df = zip_df[['zipcode',
                         'income',
                         'value',
                         'density',
                         'houses',
                         'land',
                         'irradiance',
                         'e_price',
                         'avg_contractor_wage']].rename(columns = {'income':'med_income',
                                                                   'value':'med_home_value',
                                                                   'density':'pop_density',
                                                                   'houses':'home_count',
                                                                   'land':'land_area',
                                                                   'irradiance': 'NREL_PSM_2019', 
                                                                   'e_price': 'cost_electricity'}
                                                       )


zip_cluster_df[['med_income', 
                'med_home_value', 
                'pop_density', 
                'home_count',
                'land_area', 
                'NREL_PSM_2019', 
                'cost_electricity',
                'avg_contractor_wage']] = zip_cluster_df[['med_income', 
                                                          'med_home_value',
                                                          'pop_density', 
                                                          'home_count',
                                                          'land_area', 
                                                          'NREL_PSM_2019', 
                                                          'cost_electricity',
                                                          'avg_contractor_wage']].astype(float)
zip_cluster_df.head()

,zipcode,med_income,med_home_value,pop_density,home_count,land_area,NREL_PSM_2019,cost_electricity,avg_contractor_wage
0,01001,58733.0,213000.0,1466.0,7557.0,11.44,4.575282,21.92,65846.0
1,01002,54422.0,338900.0,528.0,10388.0,55.04,4.593613,21.92,53390.0
2,01003,NaN,NaN,14581.0,5.0,0.71,NaN,21.92,53390.0
3,01005,68644.0,208500.0,115.0,2044.0,44.24,4.677823,21.92,66248.0
4,01007,71875.0,260000.0,278.0,5839.0,52.64,4.559018,21.92,53390.0


In [71]:
zip_cluster_df_full = zip_cluster_df.dropna()

Uncomment the following cell to get a look at irradiance by zip code, a measure of available solar energy.

In [72]:
# import plotly.express as px

# fig = px.choropleth_mapbox(zip_cluster_df_full, geojson=zips, locations='zipcode', color='NREL_PSM_2019',
#                            color_continuous_scale="Viridis",
#                            mapbox_style="carto-positron",
#                            zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
#                            opacity=0.5,
#                            labels={'NREL_PSM_2019':'PSM'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [73]:
zip_cluster_df_full['housing_density'] = zip_cluster_df_full.home_count.values/zip_cluster_df_full.land_area.values

In [74]:
zc_f = zip_cluster_df_full[['zipcode','med_income','med_home_value','home_count', 'pop_density', 
                            'NREL_PSM_2019', 'cost_electricity']]

In [77]:
zc_f.to_pickle(cfg["data_dir"] + cfg["zip_stats"])

In [ ]:
# from collections import defaultdict
# z_map = defaultdict(dict)

# for a,b,c,d,e,f,g in zip(zc_f.zipcode,zc_f.med_income,zc_f.med_home_value,
#                      zc_f.home_count, zc_f.pop_density, zc_f.NREL_PSM_2019, zc_f.cost_electricity):
#     z_map[a] = [b,c,d,e,f,g]
    
# z_map = dict(z_map)

In [91]:


# with open('../data/zip_features.p', 'wb') as fp:
#     pickle.dump(z_map, fp, protocol=pickle.HIGHEST_PROTOCOL)
